In [1]:
from mnist import MNIST
import numpy as np
from scipy import linalg
import random
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
mndata = MNIST('./data/')
x_train, y_train = map(np.array, mndata.load_training())
x_test, y_test = map(np.array, mndata.load_testing())
x_train = torch.FloatTensor(x_train/255.0)
y_train = torch.tensor(y_train)
x_test = torch.FloatTensor(x_test/255.0)
y_test = torch.tensor(y_test)


In [3]:
def init_W():
    d = 784
    h = 64
    k = 10
    alpha1 = (1/d)**0.5
    alpha2 = (1/h)**0.5
    W_0 = np.random.uniform(-alpha1, alpha1,(h,d))
    W_1 = np.random.uniform(-alpha2, alpha2,(k,h))
    b_0 = np.zeros((h,1))
    b_1 = np.zeros((k,1))
    W_0 = torch.FloatTensor(W_0).requires_grad_(True)
    W_1 = torch.FloatTensor(W_1).requires_grad_(True)
    b_0 = torch.FloatTensor(b_0).requires_grad_(True)
    b_1 = torch.FloatTensor(b_1).requires_grad_(True)
    return W_0,W_1,b_0,b_1

In [4]:
def fit_gd_cross_entropy(x_train, y_train,x_test,y_test):
    W_0,W_1,b_0,b_1 = init_W()
    loss_prev = 0
    d_loss = 1
    loss = 1
    accuracy = 0
    ep = 0
    batchsize = 1000
    nbatch = int(x_train.shape[0]/batchsize)
    train_accuracy_list = []
    test_accuracy_list = []
    train_loss_list = []
    test_loss_list = []
    traccuracy = 0
    taccuracy = 0
    train_loss_list = []
    while traccuracy <0.99:
        ep = ep+1
        accuracy_list = []
        for i in range(0, nbatch):
            x = x_train[i*batchsize:(i+1)*batchsize]
            y = y_train[i*batchsize:(i+1)*batchsize]
            optim = torch.optim.Adam([W_0,W_1,b_0,b_1], lr=1e-3)
            x_1 = torch.matmul(W_0, x.T) + b_0
            x_1 = torch.nn.functional.relu(x_1)
            y_hat = torch.matmul(W_1, x_1) + b_1
            #y_hat = torch.nn.functional.relu(y_hat)
            y_hat = y_hat.T
            loss = torch.nn.functional.cross_entropy(y_hat, y.long())
            optim.zero_grad()
            loss.backward()
            accuracy = float(torch.sum(torch.argmax(y_hat, axis=1) == y)) / float(y.shape[0])
            #accuracy_list.append(accuracy)
            optim.step()
        xtr_1 = torch.matmul(W_0, x_train.T) + b_0
        xtr_1 = torch.nn.functional.relu(xtr_1)
        ytr_hat = torch.matmul(W_1, xtr_1) + b_1
        #yt_hat = torch.nn.functional.relu(yt_hat)
        ytr_hat = ytr_hat.T
        train_loss = torch.nn.functional.cross_entropy(ytr_hat, y_train.long())
        train_loss_list.append(train_loss)
        traccuracy = float(torch.sum(torch.argmax(ytr_hat, axis=1) == y_train)) / float(y_train.shape[0])
        xt_1 = torch.matmul(W_0, x_test.T) + b_0
        xt_1 = torch.nn.functional.relu(xt_1)
        yt_hat = torch.matmul(W_1, xt_1) + b_1
        #yt_hat = torch.nn.functional.relu(yt_hat)
        yt_hat = yt_hat.T
        test_loss = torch.nn.functional.cross_entropy(yt_hat, y_test.long())
        test_loss_list.append(test_loss)
        #print(test_loss)
        taccuracy = float(torch.sum(torch.argmax(yt_hat, axis=1) == y_test)) / float(y_test.shape[0])
        train_accuracy_list.append(traccuracy)
        test_accuracy_list.append(taccuracy)
        print("epoch {}, accurracy on training dataset{}, accuracy on test dataset{}， loss on training dataset{}, loss on test dataset{}  ".format(ep,traccuracy,taccuracy,train_loss,test_loss))    
    print("Finally epoch {}, accurracy on training dataset{}, accuracy on test dataset{} ".format(ep,traccuracy,taccuracy))        
    return(train_accuracy_list, test_accuracy_list,ep,train_loss_list,test_loss_list)



In [26]:
train_accuracy_list, test_accuracy_list,ep ,train_loss_list,test_loss_list= fit_gd_cross_entropy(x_train, y_train,x_test,y_test)

epoch 1, accurracy on training dataset0.87245, accuracy on test dataset0.8805， loss on training dataset0.5077205300331116, loss on test dataset0.4903803765773773  
epoch 2, accurracy on training dataset0.9024, accuracy on test dataset0.907， loss on training dataset0.35048845410346985, loss on test dataset0.3390342593193054  
epoch 3, accurracy on training dataset0.9136166666666666, accuracy on test dataset0.9167， loss on training dataset0.30396756529808044, loss on test dataset0.2965088486671448  
epoch 4, accurracy on training dataset0.9208666666666666, accuracy on test dataset0.9211， loss on training dataset0.2776532471179962, loss on test dataset0.2732560634613037  
epoch 5, accurracy on training dataset0.9259666666666667, accuracy on test dataset0.9277， loss on training dataset0.25817638635635376, loss on test dataset0.2562885582447052  
epoch 6, accurracy on training dataset0.93085, accuracy on test dataset0.9311， loss on training dataset0.24151943624019623, loss on test dataset0.

epoch 49, accurracy on training dataset0.9823333333333333, accuracy on test dataset0.968， loss on training dataset0.06350728124380112, loss on test dataset0.12184586375951767  
epoch 50, accurracy on training dataset0.9824333333333334, accuracy on test dataset0.9683， loss on training dataset0.0627574697136879, loss on test dataset0.122222900390625  
epoch 51, accurracy on training dataset0.9827833333333333, accuracy on test dataset0.9681， loss on training dataset0.062033142894506454, loss on test dataset0.1226378083229065  
epoch 52, accurracy on training dataset0.9828833333333333, accuracy on test dataset0.9679， loss on training dataset0.06132935360074043, loss on test dataset0.12300563603639603  
epoch 53, accurracy on training dataset0.98335, accuracy on test dataset0.9684， loss on training dataset0.06055585667490959, loss on test dataset0.12334411591291428  
epoch 54, accurracy on training dataset0.98355, accuracy on test dataset0.9685， loss on training dataset0.05984031409025192, 

epoch 97, accurracy on training dataset0.9883, accuracy on test dataset0.9669， loss on training dataset0.04337764158844948, loss on test dataset0.16707034409046173  
epoch 98, accurracy on training dataset0.9885333333333334, accuracy on test dataset0.967， loss on training dataset0.043065741658210754, loss on test dataset0.16833870112895966  
epoch 99, accurracy on training dataset0.9885166666666667, accuracy on test dataset0.967， loss on training dataset0.04271789640188217, loss on test dataset0.16922391951084137  
epoch 100, accurracy on training dataset0.9886833333333334, accuracy on test dataset0.9671， loss on training dataset0.0423758439719677, loss on test dataset0.17008425295352936  
epoch 101, accurracy on training dataset0.9889166666666667, accuracy on test dataset0.9669， loss on training dataset0.042089398950338364, loss on test dataset0.17119954526424408  
epoch 102, accurracy on training dataset0.98895, accuracy on test dataset0.9665， loss on training dataset0.04189297184348

In [5]:
def plot(train_accuracy_list,test_accuracy_list,ep,plot_name,x_test,y_test):
    sns.set()
    plt.plot(range(1,ep+1),train_accuracy_list,'-',label ='train accuracy')
    plt.plot(range(1,ep+1),test_accuracy_list,'-',label ='test accuracy')
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.ylim(0,1)
    plt.savefig('./'+plot_name)

In [9]:
def init_W2():
    d = 784
    h0 = 32
    h1 = 32
    k = 10
    alpha1 = (1/d)**0.5
    alpha2 = (1/h0)**0.5
    alpha3 = (1/h1)**0.5
    W_0 = np.random.uniform(-alpha1, alpha1,(h0,d))
    W_1 = np.random.uniform(-alpha2, alpha2,(h1,h0))
    W_2 = np.random.uniform(-alpha3, alpha3,(k,h1))
    b_0 = np.zeros((h0,1))
    b_1 = np.zeros((h1,1))
    b_2 = np.zeros((k,1))
    W_0 = torch.FloatTensor(W_0).requires_grad_(True)
    W_1 = torch.FloatTensor(W_1).requires_grad_(True)
    W_2 = torch.FloatTensor(W_2).requires_grad_(True)
    b_2 = torch.FloatTensor(b_2).requires_grad_(True)
    b_0 = torch.FloatTensor(b_0).requires_grad_(True)
    b_1 = torch.FloatTensor(b_1).requires_grad_(True)
    
    return W_0,W_1,W_2,b_0,b_1,b_2

In [14]:
def fit_gd_cross_entropy2(x_train, y_train,x_test,y_test):
    W_0,W_1,W_2,b_0,b_1,b_2 = init_W2()
    loss_prev = 0
    d_loss = 1
    loss = 1
    accuracy = 0
    ep = 0
    batchsize = 1000
    nbatch = int(x_train.shape[0]/batchsize)
    train_accuracy_list = []
    test_accuracy_list = []
    traccuracy = 0
    taccuracy = 0
    while traccuracy <0.99:
        ep = ep+1
        accuracy_list = []
        for i in range(0, nbatch):
            x = x_train[i*batchsize:(i+1)*batchsize]
            y = y_train[i*batchsize:(i+1)*batchsize]
            optim = torch.optim.Adam([W_0,W_1,b_0,b_1], lr=1e-2)
            x_1 = torch.matmul(W_0, x.T) + b_0
            x_1 = torch.nn.functional.relu(x_1)
            x_2 = torch.matmul(W_1, x_1) + b_1
            x_2 = torch.nn.functional.relu(x_2)
            y_hat = torch.matmul(W_2, x_2) + b_2
            #y_hat = torch.nn.functional.relu(y_hat)
            y_hat = y_hat.T
            loss = torch.nn.functional.cross_entropy(y_hat, y.long())
            optim.zero_grad()
            loss.backward()
            accuracy = float(torch.sum(torch.argmax(y_hat, axis=1) == y)) / float(y.shape[0])
            #accuracy_list.append(accuracy)
            optim.step()
        xtr_1 = torch.matmul(W_0, x_train.T) + b_0
        xtr_1 = torch.nn.functional.relu(xtr_1)
        xtr_2 = torch.matmul(W_1, xtr_1) + b_1
        xtr_2 = torch.nn.functional.relu(xtr_2)
        ytr_hat = torch.matmul(W_2, xtr_2) + b_2
        #yt_hat = torch.nn.functional.relu(yt_hat)
        ytr_hat = ytr_hat.T
        traccuracy = float(torch.sum(torch.argmax(ytr_hat, axis=1) == y_train)) / float(y_train.shape[0])
        xt_1 = torch.matmul(W_0, x_test.T) + b_0
        xt_1 = torch.nn.functional.relu(xt_1)
        xt_2 = torch.matmul(W_1, xt_1) + b_1
        xt_2 = torch.nn.functional.relu(xt_2)
        yt_hat = torch.matmul(W_2, xt_2) + b_2
        #yt_hat = torch.nn.functional.relu(yt_hat)
        yt_hat = yt_hat.T
        taccuracy = float(torch.sum(torch.argmax(yt_hat, axis=1) == y_test)) / float(y_test.shape[0])
        train_accuracy_list.append(traccuracy)
        test_accuracy_list.append(taccuracy)
        #print("epoch {}, accurracy on training dataset{}, accuracy on test dataset{} ".format(ep,traccuracy,taccuracy))    
    print("Finally epoch {}, accurracy on training dataset{}, accuracy on test dataset{} ".format(ep,traccuracy,taccuracy))        
    return(train_accuracy_list, test_accuracy_list,ep)



In [15]:
train_accuracy_list, test_accuracy_list,ep = fit_gd_cross_entropy2(x_train, y_train,x_test,y_test)

epoch 1, accurracy on training dataset0.82695, accuracy on test dataset0.8369 
epoch 2, accurracy on training dataset0.89, accuracy on test dataset0.8947 
epoch 3, accurracy on training dataset0.9152, accuracy on test dataset0.916 
epoch 4, accurracy on training dataset0.9252166666666667, accuracy on test dataset0.924 
epoch 5, accurracy on training dataset0.9329, accuracy on test dataset0.9288 
epoch 6, accurracy on training dataset0.9372333333333334, accuracy on test dataset0.9338 
epoch 7, accurracy on training dataset0.9454, accuracy on test dataset0.9412 
epoch 8, accurracy on training dataset0.9473333333333334, accuracy on test dataset0.942 
epoch 9, accurracy on training dataset0.95145, accuracy on test dataset0.9471 
epoch 10, accurracy on training dataset0.95615, accuracy on test dataset0.9506 
epoch 11, accurracy on training dataset0.9557166666666667, accuracy on test dataset0.9473 
epoch 12, accurracy on training dataset0.9561333333333333, accuracy on test dataset0.9488 
epo

epoch 97, accurracy on training dataset0.9819666666666667, accuracy on test dataset0.9554 
epoch 98, accurracy on training dataset0.9824, accuracy on test dataset0.9565 
epoch 99, accurracy on training dataset0.9828166666666667, accuracy on test dataset0.9559 
epoch 100, accurracy on training dataset0.9807833333333333, accuracy on test dataset0.955 
epoch 101, accurracy on training dataset0.9822666666666666, accuracy on test dataset0.9571 
epoch 102, accurracy on training dataset0.98225, accuracy on test dataset0.9545 
epoch 103, accurracy on training dataset0.9812166666666666, accuracy on test dataset0.955 
epoch 104, accurracy on training dataset0.9824666666666667, accuracy on test dataset0.9558 
epoch 105, accurracy on training dataset0.9808166666666667, accuracy on test dataset0.9546 
epoch 106, accurracy on training dataset0.9816666666666667, accuracy on test dataset0.9561 
epoch 107, accurracy on training dataset0.9801, accuracy on test dataset0.953 
epoch 108, accurracy on train

epoch 190, accurracy on training dataset0.9851833333333333, accuracy on test dataset0.9501 
epoch 191, accurracy on training dataset0.9856333333333334, accuracy on test dataset0.9516 
epoch 192, accurracy on training dataset0.9869666666666667, accuracy on test dataset0.951 
epoch 193, accurracy on training dataset0.9872333333333333, accuracy on test dataset0.9515 
epoch 194, accurracy on training dataset0.9861, accuracy on test dataset0.9499 
epoch 195, accurracy on training dataset0.9863333333333333, accuracy on test dataset0.9516 
epoch 196, accurracy on training dataset0.9859, accuracy on test dataset0.9518 
epoch 197, accurracy on training dataset0.9847333333333333, accuracy on test dataset0.9529 
epoch 198, accurracy on training dataset0.98565, accuracy on test dataset0.9521 
epoch 199, accurracy on training dataset0.9867833333333333, accuracy on test dataset0.9522 
epoch 200, accurracy on training dataset0.9879333333333333, accuracy on test dataset0.9519 
epoch 201, accurracy on 

epoch 283, accurracy on training dataset0.9870833333333333, accuracy on test dataset0.9547 
epoch 284, accurracy on training dataset0.9879833333333333, accuracy on test dataset0.9565 
epoch 285, accurracy on training dataset0.9867, accuracy on test dataset0.9547 
epoch 286, accurracy on training dataset0.9851333333333333, accuracy on test dataset0.9543 
epoch 287, accurracy on training dataset0.9878, accuracy on test dataset0.9546 
epoch 288, accurracy on training dataset0.9875833333333334, accuracy on test dataset0.9546 
epoch 289, accurracy on training dataset0.9856166666666667, accuracy on test dataset0.9543 
epoch 290, accurracy on training dataset0.9858833333333333, accuracy on test dataset0.9551 
epoch 291, accurracy on training dataset0.9841666666666666, accuracy on test dataset0.9543 
epoch 292, accurracy on training dataset0.9862666666666666, accuracy on test dataset0.9557 
epoch 293, accurracy on training dataset0.9863833333333333, accuracy on test dataset0.9554 
epoch 294, a

epoch 376, accurracy on training dataset0.9851166666666666, accuracy on test dataset0.9507 
epoch 377, accurracy on training dataset0.98455, accuracy on test dataset0.9498 
epoch 378, accurracy on training dataset0.9832666666666666, accuracy on test dataset0.95 
epoch 379, accurracy on training dataset0.9852166666666666, accuracy on test dataset0.9506 
epoch 380, accurracy on training dataset0.98565, accuracy on test dataset0.9505 
epoch 381, accurracy on training dataset0.9863666666666666, accuracy on test dataset0.9516 
epoch 382, accurracy on training dataset0.98565, accuracy on test dataset0.9503 
epoch 383, accurracy on training dataset0.98765, accuracy on test dataset0.9514 
epoch 384, accurracy on training dataset0.9869166666666667, accuracy on test dataset0.9515 
epoch 385, accurracy on training dataset0.9859166666666667, accuracy on test dataset0.9504 
epoch 386, accurracy on training dataset0.9854, accuracy on test dataset0.9501 
epoch 387, accurracy on training dataset0.9855